## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

In [6]:
import glob
import csv
from tqdm.auto import tqdm

from entity_embed.data_utils.utils import Enumerator

row_dict = {}
id_enumerator = Enumerator()
left_id_set = set()
right_id_set = set()
rows_total = 1363 + 3226
clusters_total = 1300

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/amazon_google_exp_data/tableA.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'left-{int(row["id"])}']
            row['source'] = 'left'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/amazon_google_exp_data/tableB.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'right-{int(row["id"])}']
            row['source'] = 'right'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [7]:
train_true_pair_set = set()
valid_true_pair_set = set()
test_true_pair_set = set()

with open(f'{home_dir}/Downloads/amazon_google_exp_data/train.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            train_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/amazon_google_exp_data/valid.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            valid_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/amazon_google_exp_data/test.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            test_true_pair_set.add((id_left, id_right))
        
display(('train_true_pair_set', len(train_true_pair_set)))
display(('valid_true_pair_set', len(valid_true_pair_set)))
display(('test_true_pair_set', len(test_true_pair_set)))

('train_true_pair_set', 699)

('valid_true_pair_set', 234)

('test_true_pair_set', 234)

## Preprocess

In [8]:
attr_list = ['title', 'manufacturer', 'price']

In [9]:
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'title': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [13]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

17:52:37 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:52:40 INFO:For attr='manufacturer', computing actual alphabet and max_str_len
17:52:40 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:52:40 INFO:For attr='manufacturer', using actual_max_str_len=16
17:52:40 INFO:For attr='price', computing actual alphabet and max_str_len
17:52:40 INFO:actual_max_str_len=11 must be pair to enable NN pooling. Updating to 12
17:52:40 INFO:For attr='price', using actual_max_str_len=12


{'title': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer=<function default_tokenizer at 0x7f114eca1ca0>, alphabet=None, max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7f11d85da4f0>),
 'manufacturer': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer=<function default_tokenizer at 0x7f114eca1ca0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None),
 'price': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=<function default_tokenizer at 0x7f114eca1ca0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 

In [14]:
from entity_embed import PairwiseDataModule

datamodule = PairwiseDataModule(
    row_dict=row_dict,
    row_numericalizer=row_numericalizer,
    pos_pair_batch_size=45,
    neg_pair_batch_size=1225,
    row_batch_size=16,
    train_true_pair_set=train_true_pair_set,
    valid_true_pair_set=valid_true_pair_set,
    test_true_pair_set=test_true_pair_set,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    use_mask=True,
    embedding_size=300
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'amzn-googl'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

17:52:40 INFO:GPU available: True, used: True
17:52:40 INFO:TPU available: None, using: 0 TPU cores
17:52:40 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

17:52:42 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 4.8 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
3.1 M     Trainable params
1.7 M     Non-trainable params
4.8 M     Total params


17:52:43 WARNING:true_pair_set has 82 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [18]:
model.blocker_net.get_signature_weights()

{'title': 0.38280779123306274,
 'manufacturer': 0.2836807370185852,
 'price': 0.33351144194602966}

## Testing

In [19]:
trainer.test(ckpt_path='best')


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_at_0.3': 0.3246753246753247,
 'test_f1_at_0.5': 0.7140495867768596,
 'test_f1_at_0.7': 0.8225469728601253,
 'test_f1_at_0.9': 0.5246913580246914,
 'test_pair_entity_ratio_at_0.3': 2.4989154013015185,
 'test_pair_entity_ratio_at_0.5': 0.8047722342733189,
 'test_pair_entity_ratio_at_0.7': 0.5314533622559653,
 'test_pair_entity_ratio_at_0.9': 0.19522776572668113,
 'test_precision_at_0.3': 0.1953125,
 'test_precision_at_0.5': 0.5822102425876011,
 'test_precision_at_0.7': 0.8040816326530612,
 'test_precision_at_0.9': 0.9444444444444444,
 'test_recall_at_0.3': 0.9615384615384616,
 'test_recall_at_0.5': 0.9230769230769231,
 'test_recall_at_0.7': 0.8418803418803419,
 'test_recall_at_0.9': 0.36324786324786323}
--------------------------------------------------------------------------------


[{'test_precision_at_0.3': 0.1953125,
  'test_recall_at_0.3': 0.9615384615384616,
  'test_f1_at_0.3': 0.3246753246753247,
  'test_pair_entity_ratio_at_0.3': 2.4989154013015185,
  'test_precision_at_0.5': 0.5822102425876011,
  'test_recall_at_0.5': 0.9230769230769231,
  'test_f1_at_0.5': 0.7140495867768596,
  'test_pair_entity_ratio_at_0.5': 0.8047722342733189,
  'test_precision_at_0.7': 0.8040816326530612,
  'test_recall_at_0.7': 0.8418803418803419,
  'test_f1_at_0.7': 0.8225469728601253,
  'test_pair_entity_ratio_at_0.7': 0.5314533622559653,
  'test_precision_at_0.9': 0.9444444444444444,
  'test_recall_at_0.9': 0.36324786324786323,
  'test_f1_at_0.9': 0.5246913580246914,
  'test_pair_entity_ratio_at_0.9': 0.19522776572668113}]

## Testing manually 

In [20]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [21]:
embedding_size = model.blocker_net.embedding_size

In [22]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [23]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 272 ms, sys: 0 ns, total: 272 ms
Wall time: 42.6 ms


In [24]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 253 ms, sys: 0 ns, total: 253 ms
Wall time: 29.7 ms


In [25]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

0.5314533622559653

In [26]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, test_true_pair_set)
precision, recall

(0.8040816326530612, 0.8418803418803419)

In [27]:
f1_score(precision, recall)

0.8225469728601253

In [28]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

48

In [29]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

37

In [30]:
cos_similarity = lambda a, b: np.dot(a, b)

In [31]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.50065726,
 {'id': 207,
  'title': 'crystal reports 11 professional full product french',
  'manufacturer': 'business objects',
  'price': '909 . 66',
  'source': 'left'},
 {'id': 2888,
  'title': 'crystal reports xi professional edition complete package',
  'manufacturer': '',
  'price': '537 . 57',
  'source': 'right'})

(0.28914705,
 {'id': 1206,
  'title': 'contentbarrier x4 10 . 4 single user ( mac )',
  'manufacturer': 'intego',
  'price': '49 . 99',
  'source': 'left'},
 {'id': 1900,
  'title': 'intego contentbarrier x4 10 . 4',
  'manufacturer': '',
  'price': '54 . 99',
  'source': 'right'})

(0.44974685,
 {'id': 143,
  'title': 'adobe acrobat distiller svr v6 - cd linux u / u 42050142 )',
  'manufacturer': 'adobe systems',
  'price': '',
  'source': 'left'},
 {'id': 3193,
  'title': 'adobe 42050142 acrobat distiller 6 lnx ret cd u / l lnx',
  'manufacturer': '',
  'price': '14967 . 47',
  'source': 'right'})

(0.48714232,
 {'id': 345,
  'title': 'portfolio media kit be syst recovery 7 . 0 win small business ed',
  'manufacturer': 'symantec',
  'price': '50 . 0',
  'source': 'left'},
 {'id': 2883,
  'title': 'symantec 11859201 be sys recovery 7 . 0 win sbs ed media cd m / l',
  'manufacturer': '',
  'price': '31 . 98',
  'source': 'right'})

(0.5754837,
 {'id': 531,
  'title': 'microsoft windows server 2003 client additional license for users - 5 pack',
  'manufacturer': 'microsoft',
  'price': '809 . 0',
  'source': 'left'},
 {'id': 3876,
  'title': 'microsoft r18 - 00908 windows server 2003 client access license 20 - pack ( per user )',
  'manufacturer': 'microsoft',
  'price': '688 . 99',
  'source': 'right'})

(0.6985181,
 {'id': 1026,
  'title': "final draft scriptwriter ' s suite [ final draft 7 & final draft av2 ]",
  'manufacturer': 'final draft',
  'price': '349 . 99',
  'source': 'left'},
 {'id': 3690,
  'title': "final - draft scriptwriter ' s suite 2 . 5 screenwriting software mac / win screen writing software",
  'manufacturer': '',
  'price': '279 . 95',
  'source': 'right'})

(0.5819381,
 {'id': 273,
  'title': 'quickbooks credit card processing kit 3 . 0',
  'manufacturer': 'intuit',
  'price': '39 . 95',
  'source': 'left'},
 {'id': 1371,
  'title': 'qb credit card kit 3 . 0',
  'manufacturer': 'intuit',
  'price': '25 . 99',
  'source': 'right'})

(0.34124422,
 {'id': 726,
  'title': 'simply put software data eliminator',
  'manufacturer': 'simply - put - software',
  'price': '',
  'source': 'left'},
 {'id': 4151,
  'title': 'simply put software llc de905 - s data elminator ( win 95 98 me nt 2000 xp )',
  'manufacturer': 'simply - put - software',
  'price': '34 . 97',
  'source': 'right'})

(0.60624635,
 {'id': 1296,
  'title': 'starry night astronomy screensaver win / mac',
  'manufacturer': 'imaginova',
  'price': '19 . 99',
  'source': 'left'},
 {'id': 2055,
  'title': 'orion / imaginova starry night screensaver',
  'manufacturer': '',
  'price': '16 . 27',
  'source': 'right'})

(0.28484002,
 {'id': 812,
  'title': 'extensis smartscale 1 - user pxe - 11433 )',
  'manufacturer': 'extensis corporation',
  'price': '',
  'source': 'left'},
 {'id': 4160,
  'title': 'onone software pxe - 11433 pxl smartscale elect 1 - user english mac / win',
  'manufacturer': 'onone software',
  'price': '158 . 99',
  'source': 'right'})